# MLSS 2019: Optimal Transport for Machine Learning

In this second practical session, we will apply OT in two different Machine Learning applications:
1. Color Transfer
2. Document Clustering

In Color Transfer, we will mainly be interested in the optimal transport plan itself, while in Document Clustering, we will be interested in the value of the Optimal Transport / Wasserstein distance.

<b>We are going to use Google Collab to run this notebook. In order to install all the necessary files run the following cells:</b>

In [ ]:
import os
!pip install --upgrade git+https://github.com/mlss-skoltech/tutorials.git#subdirectory=optimal_transport_tutorial

In [ ]:
# Check your installation by importing POT
!pip install pot
import ot

Declare ```DATA_PATH``` as a path to the data from the tutorial package

In [ ]:
import pkg_resources

DATA_PATH = pkg_resources.resource_filename('optimaltransport', 'data/')

<b>If you are running this notebook locally, make sure to clone the tutorial repository:</b>

```!pip install --upgrade git+https://github.com/mlss-skoltech/tutorials.git#subdirectory=optimal_transport_tutorial```



And install the following package:

* Install with pip: ```bash pip install pot```
* Install with conda: ```bash conda install -c conda-forge pot ```

In [ ]:
import numpy as np
import ot

## 1. Color Transfer

Given a source and a target image, the goal of color transfer is to transform the colors of the source image so that it looks similar to the target image color palette. In the end, we want to find a "color mapping", giving for each color of the source image a new color. This can be done by computing the optimal transport plan between the two images, seen as point clouds in the RGB space.

In [ ]:
# For plotting
import matplotlib.pyplot as plt
from matplotlib.pyplot import imread
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
# Load the images
I1 = imread(DATA_PATH + 'schiele.jpg').astype(np.float64) / 256
I2 = imread(DATA_PATH + 'schiele2.jpg').astype(np.float64) / 256

fig = plt.figure(figsize=(17, 30))

ax = fig.add_subplot(1, 2, 1)
ax.imshow(I1)
ax.set_title('Landscape', fontsize=25)
ax.axis('off')

ax = fig.add_subplot(1, 2, 2)
ax.imshow(I2)
ax.set_title('Portrait', fontsize=25)
ax.axis('off')

plt.show()

We will need to work with "matrices" instead of images. Since there are 3 colors, images have shape `(Width, Height, 3)`, and the corresponding matrices will have shape `(Width*Height, 3)`.

In [ ]:
def im2mat(I):
    '''Convert image I to matrix.'''
    return # TODO: reshape

def mat2im(X, shape):
    '''Convert matrix X to image with shape 'shape'.'''
    return # TODO: reshape

X1 = im2mat(I1)
X2 = im2mat(I2)

Real images have way too many different colors, so we will need to subsample them. In order to do this, we use K-means over all the colors, and keep only the computed centroids. Note that using Mini Batch K-Means will speed the computations up.

In [ ]:
from sklearn.cluster import MiniBatchKMeans

# Size of the subsampled point clouds
nbsamples = 1000

kmeans1 = # TODO: Mini Batch K-Means for X1
X1_sampled = # TODO: get the centroids

kmeans2 = # TODO: Mini Batch K-Means for X2
X2_sampled = # TODO: get the centroids

Each image is represented by its "matrix", i.e. is seen as a point cloud $X \in \mathbb{R}^{N\times3}$ in the RGB color space, identified with $\mathbb{R}^3$. 

In [ ]:
def showImageAsPointCloud(X, Y):
    '''Show the color palette associated with images X and Y.'''
    fig = plt.figure(figsize=(17,8))
    ax = fig.add_subplot(121, projection='3d')
    ax.set_xlim(0,1)
    ax.scatter(X[:,0], X[:,1], X[:,2], c=X, s=10, marker='o', alpha=0.6)
    ax.set_xlabel('R',fontsize=22)
    ax.set_xticklabels([])
    ax.set_ylim(0,1)
    ax.set_ylabel('G',fontsize=22)
    ax.set_yticklabels([])
    ax.set_zlim(0,1)
    ax.set_zlabel('B',fontsize=22)
    ax.set_zticklabels([])
    ax.set_title('Landscape Color Palette', fontsize=20)
    ax.grid('off')
    
    ax = fig.add_subplot(122, projection='3d')
    ax.set_xlim(0,1)
    ax.scatter(Y[:,0], Y[:,1], Y[:,2], c=Y, s=10, marker='o', alpha=0.6)
    ax.set_xlabel('R',fontsize=22)
    ax.set_xticklabels([])
    ax.set_ylim(0,1)
    ax.set_ylabel('G',fontsize=22)
    ax.set_yticklabels([])
    ax.set_zlim(0,1)
    ax.set_zlabel('B',fontsize=22)
    ax.set_zticklabels([])
    ax.set_title('Portrait Color Palette', fontsize=20)
    ax.grid('off')
    
    plt.show()

In [ ]:
showImageAsPointCloud(X1_sampled, X2_sampled)

In order to compute the optimal transport plans between the two point clouds, we have to compute the corresponding cost matrix. In the following, we will always consider the squared distance, _i.e._ $C_{ij} = \|X_i - Y_j\|^2$.

In [ ]:
C = # TODO: compute the cost matrix using l2 ground distance

### Landscape with Portrait colors

Here, the goal is to transfer the colors of the portrait to the landscape. We will compute the exact Optimal Transport Plan, as well as the Entropy Regularized Optimal Transport plans.

In [ ]:
regs = [0.01, 0.1, 0.5]
OT_plans = [] # Contains the OT plans for regularization strengths : 0, 0.01, 0.1, 0.5
OT_plans.append(ot.emd()) # TODO: OT plan for exact OT
for reg in regs:
    OT_plans.append() # TODO: OT plan for regularization strength = reg

In [ ]:
def colorTransfer(OT_plan, kmeans1, kmeans2, shape):
    '''Return the color-transfered image of shape "shape".'''
    return # TODO

In [ ]:
fig = plt.figure(figsize=(17, 20))

ax = fig.add_subplot(1, 2, 1)
ax.imshow(I1)
ax.set_title('Source Image', fontsize=20)
ax.axis('off')

ax = fig.add_subplot(1, 2, 2)
I = colorTransfer(OT_plans[0], kmeans1, kmeans2, I1.shape)
ax.imshow(I)
ax.set_title('Reg = 0', fontsize=20)
ax.axis('off')

plt.show()

fig = plt.figure(figsize=(17, 20))
for i in range(3):
    ax = fig.add_subplot(2, 3, i+1)
    I = colorTransfer(OT_plans[i+1], kmeans1, kmeans2, I1.shape)
    ax.imshow(I)
    ax.set_title('Reg = '+str(regs[i]), fontsize=20)
    ax.axis('off')

plt.show()

### Portait with Landscape colors

We now transfer the colors of the landscape to the portrait.

In [ ]:
C = # TODO

In [ ]:
regs = [0.01, 0.03, 0.1]
OT_plans = []
OT_plans.append(ot.emd()) # TODO
for reg in regs:
    OT_plans.append() # TODO

In [ ]:
def colorTransfer(OT_plan, kmeans1, kmeans2, shape):
    return # TODO

In [ ]:
fig = plt.figure(figsize=(17, 20))

ax = fig.add_subplot(1, 2, 1)
ax.imshow(I2)
ax.set_title('Source Image', fontsize=20)
ax.axis('off')

ax = fig.add_subplot(1, 2, 2)
I = colorTransfer(OT_plans[0], kmeans1, kmeans2, I2.shape)
ax.imshow(I)
ax.set_title('Reg = 0', fontsize=20)
ax.axis('off')

plt.show()

fig = plt.figure(figsize=(17, 20))
for i in range(3):
    ax = fig.add_subplot(2, 3, i+1)
    I = colorTransfer(OT_plans[i+1], kmeans1, kmeans2, I2.shape)
    ax.imshow(I)
    ax.set_title('Reg = '+str(regs[i]), fontsize=20)
    ax.axis('off')

plt.show()

## 2. Document Clustering

We would likle to classify several text documents. In order to do this, we will:
1. Transform each text into a point cloud
2. Compute the Optimal Transport distances between each pair of point clouds
3. Use MDS to plot the different clusters in 2 dimensions

### Load the Data and Preprocessing
We consider seven movie scenarios. We transformed each of them into a point cloud using the following steps:
1. Keep only the words among the $2.000 - 20.000$ most common words
2. Each remaining word is transformed into a $300$-dimensional vector using word2vec
3. Each word is given a weight proportional to its frequency

The variable `texts` is a list of tuples. Each tuple represents a movie, and contains two parts:
1. A matrix $X \in \mathbb{R}^{n \times 300}$ where $n$ is the number of different words, containing the position of the points
2. A vector $a \in \mathbb{R}^n$ containing the corresponding weights

In [ ]:
import pickle

with open(DATA_PATH + 'texts.pickle', 'rb') as file:
    texts = pickle.load(file)

movies = ['DUNKIRK', 'GRAVITY', 'INTERSTELLAR', 'KILL BILL VOL.1', 'KILL BILL VOL.2', 'THE MARTIAN', 'TITANIC']

### Compute the OT distances

In [ ]:
# Set regularization strength
reg = 0.1

In [ ]:
def costMatrix(i,j):
    '''Return the cost matrix C between movies number i and j.'''
    X = texts[i][0]
    Y = texts[j][0]
    
    return # TODO

In [ ]:
#this cell will take approximately 1 minute to compute in Google Colaboratory after you complete it
OT_distances = np.zeros((7,7))
# TODO: compute the OT distance (using Sinkhorn algorithm ot.sinkhorn) between all the pairs of scenarios

In [ ]:
for film in movies:
    print('The film most similar to', film, 'is', # TODO)

### Plot the MDS projection

In [ ]:
from sklearn.manifold import MDS
embedding = MDS(n_components=2, dissimilarity='precomputed')
dis = OT_distances - OT_distances[OT_distances>0].min()
np.fill_diagonal(dis, 0.)
embedding = embedding.fit(dis)
X = embedding.embedding_

import matplotlib.pyplot as plt
plt.figure(figsize=(17,6))
plt.scatter(X[:,0], X[:,1], alpha=0.)
plt.axis('equal')
plt.axis('off')
c = {'KILL BILL VOL.1':'red', 'KILL BILL VOL.2':'red', 'TITANIC':'blue', 'DUNKIRK':'blue', 'GRAVITY':'black', 'INTERSTELLAR':'black', 'THE MARTIAN':'black'}
for film in movies:
    i = movies.index(film)
    plt.gca().annotate(film, X[i], size=30, ha='center', color=c[film], weight="bold", alpha=0.7)
plt.show()